<a href="https://colab.research.google.com/github/gkdivya/EVA/blob/main/4_BackpropagationAndArchitecturalBasics/TeamNotebooks/MNIST_Architecture_with_basic_techniques.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- Add batch norm 
- Reduce the number of *parameters*
- Add GAP layer
- Transition layer
- Added dropout
- updated learning rate to 0.015 amd momentum to 0.7


In [47]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [48]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 16, 3),  # 28x28 output 26x26 RF : 3x3
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout2d(0.069),

            nn.Conv2d(16, 16, 3), # 26x26 output 24x24 RF : 5x5
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout2d(0.069),

            nn.Conv2d(16, 32, 3), # 24x24 output 22x22 RF : 7x7
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Dropout2d(0.069),
        
        )

        self.trans1 = nn.Sequential(

            nn.MaxPool2d(2, 2),  # 22x22 output - 11x11 RF 14x14

            nn.Conv2d(32, 16, 1), # 22x22 output - 11x11 RF 14x14
            nn.ReLU(),
        )

        self.conv2 =  nn.Sequential(

            nn.Conv2d(16, 16, 3), # 11x11 output - 9x9 RF 16x16
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout2d(0.069),

            nn.Conv2d(16, 32, 3),  # 9x9 output - 7x7 RF 18x18
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Dropout2d(0.069),
            
        )

        self.trans2 = nn.Sequential(

            nn.Conv2d(32, 16, 1), # 9x9 output - 7x7 RF 18x18
            nn.ReLU(),
        )

        self.conv3 = nn.Sequential(
            nn.Conv2d(16, 16, 3), # 7x7 output - 5x5 RF 20x20
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout2d(0.069),

            # nn.Conv2d(16, 16, 3), # 5x5  output - 3x3 RF 22x22
            # nn.ReLU(),
            # nn.BatchNorm2d(16),
            # nn.Dropout2d(0.069),
        )


        # self.conv4 = nn.Conv2d(32, 10, 3)  # 3x3 output - 1x1 RF 24x24
        self.avg_pool = nn.Sequential(
            nn.Conv2d(16, 10, 1),
            nn.AvgPool2d(5)
        )

    def forward(self, x):
        x = self.conv1(x)
        x = self.trans1(x)
        x = self.conv2(x)
        x = self.trans2(x)
        x = self.conv3(x)
        x = self.avg_pool(x)

        x = x.view(-1,10)
        return F.log_softmax(x,dim=1)

In [49]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 26, 26]             160
              ReLU-2           [-1, 16, 26, 26]               0
       BatchNorm2d-3           [-1, 16, 26, 26]              32
         Dropout2d-4           [-1, 16, 26, 26]               0
            Conv2d-5           [-1, 16, 24, 24]           2,320
              ReLU-6           [-1, 16, 24, 24]               0
       BatchNorm2d-7           [-1, 16, 24, 24]              32
         Dropout2d-8           [-1, 16, 24, 24]               0
            Conv2d-9           [-1, 32, 22, 22]           4,640
             ReLU-10           [-1, 32, 22, 22]               0
      BatchNorm2d-11           [-1, 32, 22, 22]              64
        Dropout2d-12           [-1, 32, 22, 22]               0
        MaxPool2d-13           [-1, 32, 11, 11]               0
           Conv2d-14           [-1, 16,

In [50]:
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.RandomRotation(5),
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [51]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'epoch={epoch} loss={loss.item():.10f} batch_id={batch_idx:05d}')
    #print( f'Epoch {epoch} - \nTrain set : loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [52]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.015, momentum=0.7)

for epoch in range(1, 20):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]

Test set: Average loss: 0.1063, Accuracy: 9750/10000 (97.50%)



  0%|          | 0/469 [00:00<?, ?it/s]

Test set: Average loss: 0.0559, Accuracy: 9841/10000 (98.41%)



  0%|          | 0/469 [00:00<?, ?it/s]

...

Test set: Average loss: 0.0203, Accuracy: 9935/10000 (99.35%)



  0%|          | 0/469 [00:00<?, ?it/s]

Test set: Average loss: 0.0192, Accuracy: 9941/10000 (99.41%)



epoch=19 loss=0.0342727378 batch_id=00468: 100%|██████████| 469/469 [00:28<00:00, 16.44it/s]


Test set: Average loss: 0.0185, Accuracy: 9948/10000 (99.48%)

